# Mesh with SALOME

## Building the geometry in SALOME

This is a step-by-step guide to meshing with [SALOME 9.12.0](https://www.salome-platform.org/>).

1. Open SALOME and create a new study.
2. Activate the Geometry module

   ```{figure} salome/salome_guide_1.png
   ---
   width: 400
   ---
   ```

3. Create a first square by clicking "Create rectangular face". Keep the default parameters. Click "Apply and Close"

   ```{figure} salome/salome_guide_2.png
   ---
   width: 400
   ---
   ```

4. Repeat the operation to create a second square

5. Translate the second square by clicking "Operations/Transformation/Translation"


   ```{figure} salome/salome_guide_3.png
   ---
   width: 400
   ---
   ```

6. Make sure Face 2 is selected. Enter 100 for the Dx value. Click "Apply and Close"

   ```{figure} salome/salome_guide_4.png
   ---
   width: 400
   ---
   ```

7. Create a compound by clicking "New Entity/Build/Compound" make sure Face_1 and Translation_1 are selected then click "Apply and Close".

   ```{figure} salome/salome_guide_5.png
   ---
   width: 400
   ---
   ```

8. Create a group "New Entity/Group/Create group". In Shape Type, select the 2D surface. Name the group "left_volume". Make sure Compound_1 is selected.
   Click on the left square and click "Add" (2 should appear in the white window). Click "Apply and Close".

   ```{figure} salome/salome_guide_6.png
   ---
   width: 400
   ---
   ```

9. Repeat the operation to create a group "right_volume" with the right square (12 should appear in the white window).

10. Create another group "left_boundary" but this time in Shape Type select the 1D curve. Click on the left edge of the left square and click "Add". Click "Apply and Close".

   ```{figure} salome/salome_guide_7.png
   ---
   width: 400
   ---
   ```

11. Repeat the operation to create a group "right_boundary" with the right edge of the right square. Your study should look like:

   ```{figure} salome/salome_guide_8.png
   ---
   width: 400
   ---
   ```

12. Click on "Mesh" to activate the mesh module.

   ```{figure} salome/salome_guide_9.png
   ---
   width: 400
   ---
   ```

13. Create a mesh by clicking "Mesh/Create Mesh".

14. Make sure Compound_1 is selected in "Geometry". Under the 2D tab, select "NETGEN 1D-2D" as algorithm.

   ```{figure} salome/salome_guide_10.png
   ---
   width: 400
   ---
   ```

15. Next to "Hypothesis" click on the gear symbol. Select "NETGEN 2D Simple Parameters". Click Ok. Click "Apply and Close".

   ```{figure} salome/salome_guide_11.png
   ---
   width: 400
   ---
   ```

    In the Objet Browser, under Mesh_1 you should see Groups of Edges and Groups of Faces, containing left_boundary, right_boundary, left_volume and right_volume.

16. Export the mesh to MED by right clicking on Mesh_1 in the Object Browser, then Export/MED file. Choose a location where you want to write your MED file and click Save.

   ```{figure} salome/salome_guide_12.png
   ---
   width: 400
   ---
   ```


## Convert the mesh to XDMF format

Using [meshio](https://github.com/nschloe/meshio), the mesh file (here a .med file) can be converted to .xdmf format, readable by FESTIM.

We wrote the following function `convert_med_to_xdmf` to convert .med files to .xdmf.

In [1]:
import meshio


def convert_med_to_xdmf(
    med_file,
    cell_file="mesh_domains.xdmf",
    facet_file="mesh_boundaries.xdmf",
    cell_type="tetra",
    facet_type="triangle",
):
    """Converts a MED mesh to XDMF
    Args:
        med_file (str): the name of the MED file
        cell_file (str, optional): the name of the file containing the
            volume markers. Defaults to "mesh_domains.xdmf".
        facet_file (str, optional): the name of the file containing the
            surface markers.. Defaults to "mesh_boundaries.xdmf".
        cell_type (str, optional): The topology of the cells. Defaults to "tetra".
        facet_type (str, optional): The topology of the facets. Defaults to "triangle".
    Returns:
        dict, dict: the correspondance dict, the cell types
    """
    msh = meshio.read(med_file)

    correspondance_dict = {-k: v for k, v in msh.cell_tags.items()}
    
    cell_data_types = msh.cell_data_dict["cell_tags"].keys()

    for mesh_block in msh.cells:
        if mesh_block.type == cell_type:
            meshio.write_points_cells(
                cell_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][cell_type]]},
            )
        elif mesh_block.type == facet_type:
            meshio.write_points_cells(
                facet_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][facet_type]]},
            )

    return correspondance_dict, cell_data_types

The subdomains (volumes and surfaces) in the .med file were given tags.

This is very important to be able to assign materials properties and boundary conditions to these subdomains.

The correspondance between tags and subdomains is returned by `convert_med_to_xdmf`.
Here for example, the volume corresponding to `tungsten` is the tag `6`.

In [2]:
correspondance_dict, cell_data_types = convert_med_to_xdmf(
    "salome/Mesh_1.med",
    cell_file="salome/mesh_domains.xdmf",
    facet_file="salome/mesh_boundaries.xdmf",
    cell_type="triangle",
    facet_type="line",
)

print(correspondance_dict)

{np.int64(6): ['left_volume'], np.int64(7): ['right_volume'], np.int64(8): ['left_boundary'], np.int64(9): ['right_boundary']}


In [3]:
import festim as F

mesh = F.MeshFromXDMF("salome/mesh_domains.xdmf", "salome/mesh_boundaries.xdmf")

In [4]:
from dolfinx import plot
import pyvista

pyvista.start_xvfb()
pyvista.set_jupyter_backend("html")

tdim = mesh.mesh.topology.dim

mesh.mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh.mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…